In [1]:
import numpy as np
from tqdm import tqdm
from constants import MAX_SEQ_LEN

with open('../hw_train.csv', 'rb') as f:
    ls = f.readlines()[1:]
# count vocabulary and max sequence length
lyrics_map = {}
bos_token = 's'.decode('utf8')
eos_token = 'e'.decode('utf8')
oov_token = 'o'.decode('utf8')


none_lyrics_words = [u'編詞', u'作曲', u'作詞', u'編曲', u'監製']
vocab = [bos_token, eos_token, oov_token]
lens = []
c = 0 
with tqdm(total=len(ls)) as pbar:
    for l in ls:
        l = l.decode('utf8')
        no, l = l.strip().split(',')
        no = int(no)
        if no not in lyrics_map:
            lyrics_map[no] = []
        lyrics_map[no].append(l)
        pbar.update(1)
# split into Q,K pair
with tqdm(total=len(lyrics_map)) as pbar:
    for k,v in lyrics_map.items():
        line_num = len(v)
        line_list = []
        for line in v:
            exist_none_layrics_word = False
            for nw in none_lyrics_words:
                if nw in line:
                    exist_none_layrics_word = True
                    break
            if exist_none_layrics_word:
                continue
            word_list = [bos_token, ]
            for w in line.strip().split(' '):
                word_list.append(w)
                vocab.append(w)
#                 
                vocab.append(oov_token) # to increase oov_token frequence
                vocab.append(bos_token) # to increase oov_token frequence
                vocab.append(eos_token) # to increase oov_token frequence
            word_list.append(eos_token)
            lens.append(len(word_list))
            line_list.append(word_list)
        lyrics_map[k] = line_list
        pbar.update(1)
            
print 'vocabulary size : %d, max len : %.0f, mean : %.0f, std : %.0f' % (len(set(vocab)), np.max(lens), np.mean(lens), np.std(lens))
print 'lyrics number : %d' % len(lyrics_map)
# print np.max(lens), np.mean(lens), np.std(lens)
p = np.array(lens) > 50
print np.sum(p), len(lens)


100%|██████████| 18338/18338 [00:02<00:00, 7469.68it/s]


vocabulary size : 92027, max len : 243, mean : 8, std : 3
lyrics number : 18338
21 667056


In [2]:
import numpy as np
import keras
import pickle
from constants import VOCAB_DIM
from keras.preprocessing.sequence import pad_sequences

texts = vocab

tokenizer = keras.preprocessing.text.Tokenizer(num_words=VOCAB_DIM, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ',
                                   lower=False, split=' ', char_level=False, oov_token=oov_token)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
# tokenizer.num_words = VOCAB_DIM
print word_index[bos_token] , word_index[oov_token] , word_index[eos_token]
assert word_index[bos_token] < VOCAB_DIM and word_index[oov_token] < VOCAB_DIM and word_index[eos_token] < VOCAB_DIM 
print('Found %s unique tokens.' % len(word_index))

with open('tokenizer','wb') as f:
    pickle.dump(tokenizer, f)


Using TensorFlow backend.


2 4 3
Found 92027 unique tokens.


In [3]:
from constants import MAX_SEQ_LEN
from keras.preprocessing.sequence import pad_sequences
from constants import MAX_SEQ_LEN

idx = np.random.permutation(len(lyrics_map))
val_num = len(lyrics_map)//10
train_idx, val_idx = idx[val_num:], idx[:val_num]
train_u_map = {k:lyrics_map[k] for k in lyrics_map.keys()[val_num:]}
val_u_map = {k:lyrics_map[k] for k in lyrics_map.keys()[:val_num]}
def batch_boostrap_generator(batch_size, lyrics_map, max_seq_len, tokenizer):
    G = boostrap_generator(lyrics_map, max_seq_len, tokenizer)
    while True:
        Q, K = [], []
        for i in range(batch_size):
            q, k = next(G)
            Q.append(q)
            K.append(k)
        yield np.vstack(Q), np.vstack(K)

# pad will be treat as oov
def boostrap_generator(lyrics_map, max_seq_len, tokenizer):
    while True:
        keys = lyrics_map.keys()
        for idx in np.random.permutation(len(lyrics_map)):
            no = keys[idx]
            line_list = lyrics_map[no]
            for i in range(len(line_list)-1):
                K, Q = line_list[i], line_list[i+1]
                Q = tokenizer.texts_to_sequences([Q,])
                Q = pad_sequences(Q, maxlen=MAX_SEQ_LEN, dtype=np.int32, padding='post', truncating='post', value=0)
                K = tokenizer.texts_to_sequences([K,])
                K = pad_sequences(K, maxlen=MAX_SEQ_LEN, dtype=np.int32, padding='pre', truncating='pre', value=0)
                yield Q, K
    
bat = 4
G = batch_boostrap_generator(bat, train_u_map, MAX_SEQ_LEN, tokenizer)
q, k = next(G)
print tokenizer.sequences_to_texts(q)
print q.shape, k.shape


[u's \u521a \u8d70\u8fdb \u793e\u4f1a \u4eba \u5f97\u7f6a \u4e00\u5806 e o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o', u's \u53d1\u578b \u91cc \u7684 \u53e6\u7c7b \u8d2b\u7a77 \u5374 \u6709 \u54c1\u5473 e o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o', u's \u8c01 \u4e5f \u5b66 \u4e0d\u4f1a e o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o', u's \u4e0d \u559c\u6b22 \u7eb9\u8eab \u9f99\u51e4 \u7cfb\u5217 e o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o']
(4, 50) (4, 50)


In [4]:
q, k = next(G)
print q.shape
print tokenizer.sequences_to_texts(q[0:1,:])

(4, 50)
[u's \u4e3a \u4e00\u4e2a \u7231 \u7684 \u4eba \u53bb \u6d41\u8840 e o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o']


### RNN

In [5]:
import torch
from Seq2seq import Seq2seq
from constants import D_MODEL, MAX_SEQ_LEN

bat = 7
lay_num = 2
model = Seq2seq(D_MODEL, lay_num)
model.cuda()
# print(o.size())
q, k = next(G)
q = torch.LongTensor(q).cuda()
k = torch.LongTensor(k).cuda()

o = model(q, k)
print(o.size())
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))



torch.Size([4, 50, 30001])
8106545


In [ ]:
import torch.nn as nn
from collections import deque
from tqdm import tqdm as tqdm

import time
def dump_log(model, n_iter, loss, val_loss, acc, val_acc, train_pred, train_label, val_pred, val_label, log_file_stream, tmp_model_path):
    log_text = '%.7d<split>%.5f<split>%.5f<split>%.5f<split>%.5f<split>%s<split>%s<split>%s<split>%s\n' % (n_iter, loss, val_loss, acc, val_acc, train_pred, train_label, val_pred, val_label)
    log_file_stream.write(log_text.encode('utf8'))
    if n_iter % 10 == 0 :
        log_file_stream.flush()
        torch.save(model, tmp_model_path)
def normal_acc(pred, label):
    pred = torch.argmax(pred, dim=-1)
    mask = torch.ones_like(label, dtype=torch.uint8)
#     
    acc = pred == label
    acc = torch.sum(acc).item() / float(torch.sum(mask).item())
    
    return acc
def seq2text(output):
    assert len(output.shape) == 2
    output = output.cpu().numpy()
    seq = tokenizer.sequences_to_texts(output)
    s = seq[0]
    s = s.replace(oov_token,u'')
    return s
    
def rev_mask(m):
    out = torch.ones_like(m, dtype=torch.uint8, requires_grad=False)
    out.masked_fill_(m, 0)
    return out
        
acc_q = deque(maxlen=100)
loss_q = deque(maxlen=10)


val_acc_q = deque(maxlen=100)
val_loss_q = deque(maxlen=10)

t = time.time()
best_loss = float('inf')

epochs = 100
batch_size = 64
G = batch_boostrap_generator(batch_size, train_u_map, MAX_SEQ_LEN, tokenizer)
val_G = batch_boostrap_generator(batch_size, val_u_map, MAX_SEQ_LEN, tokenizer)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
print 'start training.'
with open('log.txt', 'w') as f:
    with open('best.txt', 'w') as best_log:
        iters = 100000000
        with tqdm(total=iters*batch_size) as pbar:
            for it in range(iters):
                optimizer.zero_grad()
                model.train()
                q, k = next(G)
                q = torch.LongTensor(q).cuda()
                k = torch.LongTensor(k).cuda()
                q.requires_grad_(False)
                k.requires_grad_(False)
                
                
                output = model(q, k)
                y = q[:,1:]
                loss = criterion(output[:,:-1,:,].permute(0,2,1), y) 
                label = y

                pred = output[:,:-1,:,]
                acc = normal_acc(pred, label)
                acc_q.append(acc)
                train_pred = seq2text(torch.argmax(output[0:1,:,:], dim=-1))
                train_label= seq2text(y[0:1,:])
                loss.backward()
                loss_q.append(loss.item())
                
                optimizer.step()
                with torch.no_grad():
                    model.eval()
                    q, k = next(val_G)
                    a = q
                    b = k
                    q = torch.LongTensor(q).cuda()
                    k = torch.LongTensor(k).cuda()
                    
                    q.requires_grad_(False)
                    k.requires_grad_(False)
                    assert q.is_cuda and k.is_cuda
                    output = model(q, k)
                    y = q[:,1:]
                    val_loss = criterion(output[:,:-1,:,].permute(0,2,1), y) 
                    label = y

                    pred = output[:,:-1,:,]
                    val_acc = normal_acc(pred, label)
                    val_acc_q.append(val_acc)
                    val_pred = seq2text(torch.argmax(output[0:1,:,:], dim=-1))
                    val_label= seq2text(y[0:1,:])
                    val_loss_q.append(val_loss.item())


                
                acc = np.mean(acc_q)
                val_acc = np.mean(val_acc_q)
                loss = np.mean(acc_q)
                val_loss = np.mean(val_acc_q)
                
                pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, loss : %.3f, val_loss : %.3f' % (acc, val_acc, loss.item(), val_loss.item()), refresh=False)
                pbar.update(batch_size)
                dump_log(model, (it+1)*batch_size, loss, val_loss, acc, val_acc, train_pred, train_label, val_pred, val_label, f,'./tmp.pt')
                if val_loss.item() < best_loss and it > 100:
                    torch.save(model, './best.pt')
                    best_loss = val_loss
                    best_log.write('%d\t%.5f\n' % ((it+1)*batch_size, best_loss))
                    best_log.flush()
                if it % 100 == 0 and it >= 100: 
                    print 'train pred : %s\ntrain label : %s' % (train_pred, train_label)
                    print 'validation pred : %s\nvalidation label : %s' % (val_pred, val_label)

# Train model
print("Optimization Finished!")
# print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

  0%|          | 0/6400000000 [00:00<?, ?it/s]

start training.


  0%|          | 6592/6400000000 [00:08<1807:03:03, 983.80it/s, acc : 0.885, val_acc : 0.888, loss : 0.885, val_loss : 0.888] 

train pred : 我 我 的 e e e                                            
train label : 拍 在 我 的 脸颊 e                                           
validation pred : 我 我 的 e                                              
validation label : 来 煽风来 点火 e                                             


  0%|          | 12992/6400000000 [00:16<1764:52:18, 1007.31it/s, acc : 0.887, val_acc : 0.890, loss : 0.887, val_loss : 0.890]

...

  0%|          | 864256/6400000000 [14:25<1693:40:30, 1049.52it/s, acc : 0.891, val_acc : 0.891, loss : 0.891, val_loss : 0.891]

train pred : 我 的 e                                               
train label : 时空 变换 e                                              
validation pred : 我 你 是 你 e e e e e e e                                       
validation label : 说 怎么 爱 怎么 散 却 不能 放 不能 忘情 e                                      


  0%|          | 870656/6400000000 [14:31<1681:28:02, 1057.13it/s, acc : 0.892, val_acc : 0.895, loss : 0.892, val_loss : 0.895]

train pred : 我 你 的 是 我 是 e e e                                         
train label : 假如 过往 只能 让 人 故意 逃避 现在 e                                        
validation pred : 我    的                                             
validation label :  當中 只有 我 e                                            


  0%|          | 871424/6400000000 [14:32<1722:02:30, 1032.23it/s, acc : 0.893, val_acc : 0.893, loss : 0.893, val_loss : 0.893]

In [ ]:
v = seq2text(k[0:1,:])
print v
print k[0:1,:]

print q